Para executar os notebooks a seguir, se ainda não fez isso, você precisa definir a chave da OpenAI no arquivo .env como `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Em seguida, vamos carregar o Índice de Embeddings em um Dataframe do Pandas. O Índice de Embeddings está armazenado em um arquivo JSON chamado `embedding_index_3m.json`. O Índice de Embeddings contém os Embeddings para cada uma das transcrições do YouTube até o final de outubro de 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Em seguida, vamos criar uma função chamada `get_videos` que vai buscar no Índice de Embeddings pela consulta. A função vai retornar os 5 vídeos mais parecidos com a consulta. O funcionamento da função é o seguinte:

1. Primeiro, é criada uma cópia do Índice de Embeddings.
2. Depois, o Embedding para a consulta é calculado usando a API de Embedding da OpenAI.
3. Em seguida, é criada uma nova coluna no Índice de Embeddings chamada `similarity`. A coluna `similarity` contém a similaridade cosseno entre o Embedding da consulta e o Embedding de cada segmento de vídeo.
4. Depois, o Índice de Embeddings é filtrado pela coluna `similarity`. O Índice de Embeddings é filtrado para incluir apenas vídeos que tenham similaridade cosseno maior ou igual a 0,75.
5. Por fim, o Índice de Embeddings é ordenado pela coluna `similarity` e os 5 vídeos mais relevantes são retornados.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Esta função é muito simples, ela apenas exibe os resultados da consulta de pesquisa.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Primeiro, o Índice de Embeddings é carregado em um Dataframe do Pandas.
2. Em seguida, o usuário é solicitado a digitar uma consulta.
3. Depois, a função `get_videos` é chamada para buscar a consulta no Índice de Embeddings.
4. Por fim, a função `display_results` é chamada para mostrar os resultados ao usuário.
5. O usuário então é solicitado a digitar outra consulta. Esse processo continua até que o usuário digite `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.br.png)

Você será solicitado a digitar uma consulta. Digite uma consulta e pressione enter. O aplicativo vai retornar uma lista de vídeos que são relevantes para a consulta. O aplicativo também vai mostrar um link para o ponto do vídeo onde está a resposta para a pergunta.

Aqui estão algumas consultas para testar:

- O que é Azure Machine Learning?
- Como funcionam as redes neurais convolucionais?
- O que é uma rede neural?
- Posso usar Jupyter Notebooks com Azure Machine Learning?
- O que é ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte autorizada. Para informações críticas, recomenda-se a tradução profissional feita por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes do uso desta tradução.
